In [120]:
import math
import cv2
import glob
from skimage.transform import resize
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from copy import deepcopy

In [3]:
def erosion(image, n, m):
    result = np.zeros((np.shape(image)[0],(np.shape(image)[1])))
    for i in range(np.shape(image)[0] - n):
        for j in range(np.shape(image)[1] - m):
            result[i+math.floor(n/2)][j+math.floor(m/2)] = np.min(image[i:i+n-1, j:j+m-1])
            
    return result

In [110]:
def dilation(image, n, m):
    result = np.zeros((np.shape(image)[0],(np.shape(image)[1])))
    for i in range(np.shape(image)[0] - n):
            result[i+math.floor(n/2)] = image[i+math.floor(n/2)]
            
    return result

## Blob detection

In [147]:
def bounding(label_name, seg_name):
    labels = np.load('./labels/'+label_name)['arr_0']
    labels = np.round(resize(labels, (1088, 1920))).astype(np.uint8)
    labels = labels*255
    seg_img = cv2.imread('./labels/'+seg_name)
    kernel = np.ones((40,40), np.uint8) 
    image_gray = cv2.erode(labels, kernel, iterations=1) 
    kernel = np.ones((30,30), np.uint8) 
    image_gray = np.array(cv2.dilate(image_gray, kernel, iterations=1))
    image_gray = dilation(image_gray, 2,2)
    image_gray = image_gray.astype(float)
    c=image_gray[1][1]
    img = image_gray
    # Set up the detector with default parameters. 
    img = (img).astype('uint8')
    th, threshed = cv2.threshold(img, 200, 255, 
           cv2.THRESH_BINARY_INV) 
    
    cnts = cv2.findContours(threshed, cv2.RETR_LIST, 
                        cv2.CHAIN_APPROX_NONE )[-2] 
    xcnts = [] 
    for cnt in cnts: 
        xcnts.append(cnt) 

    for c in cnts:
        # get the bounding rect
        x, y, w, h = cv2.boundingRect(c)
        # draw a white rectangle to visualize the bounding rect
        cv2.rectangle(seg_img, (x-50, y-50), (x + w+50, y + h+50), 0, 2)
    cv2.imwrite('./bounded/box' + seg_name, seg_img)

    centers=[]
    for xcnt in xcnts:
        min_values = np.min(xcnt,axis=0)[0]
        max_values = np.max(xcnt,axis=0)[0]
        center_x = (min_values[0] + max_values[0])/2
        center_y = (min_values[1] + max_values[1])/2   
        center = [center_x, center_y]
        centers.append(center)


    return centers


## Tracking algorithm

In [ ]:
nr_of_files = 0
file = sorted(glob.glob('./labels/*.png'))
seg_name = 'frame0000.png'
label_name = 'labels_0000.npz'
old_centers = bounding(label_name, seg_name)
img = Image.open('./bounded/box'+ seg_name)
draw = ImageDraw.Draw(img)
font = ImageFont.truetype('theboldfont.ttf', 20)

centers_dict = {}

tag_count = 0

for idx,center in enumerate(old_centers):
    print(tag_count)
    tag_count+=1
    draw.text((center[0]-60, center[1]),str(idx),(0,0,0), font=font) # this will draw text with Blackcolor and 16 size
    centers_dict[idx] = str(idx)
img.save('./bounded/box'+seg_name)
contor = 0
for idx in range(1, np.shape(file)[0]):
    new_centers_dict = {}
    
    seg_name = 'frame' + str(idx).zfill(4)+'.png'
    label_name = 'labels_' + str(idx).zfill(4)+'.npz'
    new_centers = bounding(label_name, seg_name)
    
    img = Image.open('./bounded/box'+ seg_name)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype('theboldfont.ttf', 20)
    used_idx = []
    
    for new_idx, new_center in enumerate(new_centers):
        min_dist = 100000
        min_idx = -1
        for old_idx, old_center in enumerate(old_centers) :
            dist = math.sqrt((old_center[0] - new_center[0])**2 + (old_center[1] - new_center[1])**2)
            if min_dist > dist and old_idx not in used_idx:
                min_dist = dist
                min_idx = old_idx
                
        if min_dist < 60 :
            tag_name = centers_dict[min_idx]
            used_idx.append(min_idx)
            
            centers_dict.pop(min_idx, None)
            new_centers_dict[new_idx] = tag_name
        else :
            new_centers_dict[new_idx] = str(tag_count)
            tag_count += 1
        
        draw.text((new_center[0]-60, new_center[1]),str(new_centers_dict[new_idx]),(0,0,0), font=font) # this will draw text with Blackcolor and 16 size
        
    old_centers = deepcopy(new_centers)
    centers_dict = deepcopy(new_centers_dict)
    img.save('./bounded/box'+seg_name)
    contor += 1
    if contor %100 ==0:
        print(contor)
